<a href="https://colab.research.google.com/github/ZoeBerling/AlgorithmsRepositoryCOMP4581/blob/main/newSalesTrends.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import plotly.graph_objects as go
import numpy as np
import os
import datetime
from datetime import date 
import plotly.express as px
import plotly
import warnings

from dateutil import parser

warnings.filterwarnings("ignore")
from io import SEEK_CUR

from datetime import date
from google.colab import auth
auth.authenticate_user()

import gspread
from oauth2client.client import GoogleCredentials

from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [15]:
def all_orders(directory, years):
  # """"open all_order repot txts saved in the google drive folder"""
  imp_columns = ['amazon-order-id','merchant-order-id','sales-channel', 'order-status','purchase-date', 
               # 'date', 
               'sku', 'promotion-ids', 'asin', 'quantity', 'ship-state']
  allorders = pd.DataFrame()
  for year in years:
      for root,dirs,files in os.walk(f'{directory}/{year}'):
          for file in files:
              if file.endswith(".txt"):
                  # print(year, file)
                  allorders = allorders.append(pd.read_csv(
                      f'{directory}/{year}/{file}', sep = '	'), ignore_index=True)
            
  allorders = allorders[imp_columns]          
  allorders['date']= pd.to_datetime(allorders['purchase-date'], errors='coerce', utc=True)
  print("data import complete")
  return allorders

def cull(df_sales, channel):
  # Select only Amazon or Shopify sales (replacements are counted as sales)
  sales = [channel]
  df_amz = df_sales[df_sales['sales-channel'].str.contains('|'.join(sales))]  # both of these work the same way
  shopify = df_sales[
                  (df_sales['merchant-order-id'].str.len()==7) &
                    (df_sales['merchant-order-id'].str.contains('|'.join('#')))]
  return df_amz.append(shopify, ignore_index=True)

def day(df):
  # sum by day
  cum_all_day = df.groupby([pd.Grouper(key='date', freq = 'D'), pd.Grouper('sku')])['quantity'].sum().to_frame()
  cum_all_day.reset_index(inplace=True)
  cum_all_day.set_index('date', inplace=True)
  return cum_all_day

def cum(mydf, i_freq = [7, 30, 90, 180]):
  # using day(), sum by defined frequency take in df and list of summation period frequencies: return sum by period, mean by period, and weighted averages
  df = day(mydf)
  byday = df.copy()
  byday.reset_index(inplace=True)
  # I think I can put the landscape sheet stuff here? I just need to figure out how to concat on sku: but the index is reset?

  byday.set_index(['date', 'sku'], inplace=True)
  byday.columns =  ['daily']

  for i in i_freq:

    mydf = df.groupby('sku')['quantity'].rolling(f'{i}D').sum().to_frame()
    mydf.reset_index(inplace=True)
    mydf.set_index(['date', 'sku'], inplace=True)
    mydf.columns = [f'sum{i}days']
    # display(mydf.head())

    mydf2 = df.groupby('sku')['quantity'].rolling(f'{i}D').mean().to_frame()
    mydf2.reset_index(inplace=True)
    mydf2.set_index(['date', 'sku'], inplace=True)
    mydf2.columns = [f'avg{i}']

    byday = byday.join([mydf2, mydf], how='outer')

  byday.fillna(0, inplace=True)
  #  complete_cum['weighted_avg_daily_sales'] = ((complete_cum['daily']*.3) + ((complete_cum['weekly']/7)*.3) + ((complete_cum['monthly']/30)*.3) + ((complete_cum['6_mon']/(30*6))*.1))
  byday['weighted_avg_weekly_sales'] = (((byday['avg7'])*.25) + ((byday['avg30'])*.30) + ((byday['avg90'])*.25) + ((byday['avg180'])*.20))
  display(byday.head(2))
  
  return byday

"""get minimum and maximum and return dataframe with sku and min and max"""
def get_min_max(df):
  
  mydf = df.copy()
  date_df = mydf.iloc[-180:]

  date_df.reset_index(inplace=True)

  date_df.set_index('date', inplace = True)
  date_df['weekday'] = date_df.index.weekday

  new = date_df.groupby(['sku']).agg({'sum7days': ['min', 'max']})
  return(new)

  
def create_report(mydf, time_d=1):
  # minmax = get_min_max(mydf)
  df = mydf.copy()
  df.reset_index(inplace=True)
  df.set_index('date', inplace=True)
  today = datetime.date.today() 
  pastday = today - datetime.timedelta(days=time_d)  # default is yesterday assuming folder was updated today
  date = str(pastday)
  finaldf = df.loc[date]
  # finaldf.sort_values('weekdiff', ascending=True, inplace=True)
  finaldf.drop('daily', axis=1, inplace=True)
  finaldf.reset_index(inplace = True)
 
  # finaldf = minmax.merge(finaldf, on='sku', left_index=True)

  return finaldf
  
def create_sheet(df, url, time_d=0, mytitle='default_title'):
  today = datetime.date.today() 
  pastday = today - datetime.timedelta(days=time_d)
  gc = gspread.authorize(GoogleCredentials.get_application_default())

  # Open our new sheet and add some data.
  myurl = url
  
  # myurl = 'https://docs.google.com/spreadsheets/d/1nRCa_ZGTVjmXT1kM1FaFVooBji_SV2zhQ8L-2UbOs84/edit#gid=0'  # this is my test sheet
  wb = gc.open_by_url(myurl)
  mytitle = f'{pastday}{mytitle}'
  try:
    sheet = wb.add_worksheet(title=mytitle, rows="100", cols="20")
    print(f'Sheet {pastday}{mytitle}')
  except:
    print('Updating existing worksheet: please check to make sure values are correct. If not, delete sheet and run again')
    sheet = wb.worksheet(mytitle)


  r = f'A1:D{len(df)}'
  cell_list = sheet.range(r)

  from gspread_dataframe import set_with_dataframe

  set_with_dataframe(sheet,df, include_index=True)

def get_min_max(df):
  """Takes in the complete_cum data (create report)"""
  
  mydf = df.copy()
  date_df = mydf.iloc[-180:]
  date_df.reset_index(inplace=True)
  date_df.set_index('date', inplace = True)
  date_df['weekday'] = date_df.index.weekday
  new = date_df.groupby(['sku']).agg({'sum7days': ['min', 'max']})
  return(new)


def landscape(mydf):
  """Takes in xxdf"""
  mydf.sort_values('date', inplace=True)
  mydf['week_delta'] = pd.to_datetime(mydf['date']) - pd.to_timedelta(6, unit='d') # set time delta

  cum_weekly = mydf.groupby('sku').resample('W', on='week_delta').sum()
  # cum_weekly.set_index(['sku', 'week_delta'],inplace=True)

  cum_weekly.reset_index(inplace=True)
  cum_weekly['week_delta'] = cum_weekly['week_delta'].dt.date

  weekly_df=cum_weekly.pivot(index='sku', columns='week_delta', values='quantity')

  weekly_df = weekly_df.iloc[: ,-2 :-1]

  return weekly_df



"""Test complete flow"""

usdir = '/content/drive/My Drive/Sales_Data_Connected_Folder/Amz_Data/ALL_ORDERS'
y = ['2017', '2018', '2019', '2020', '2021', '2022']
usdf = all_orders(usdir, y)
usdf = cull(usdf, 'Amazon.com')
usdf.sort_values('date', ascending=False, inplace=True)
us_complete_df = cum(usdf)

last_week  = landscape(usdf)
min_max = get_min_max(us_complete_df)
my_range = last_week.merge(min_max, on='sku')

repo_us = create_report(us_complete_df, 1)  # default is 1
repo_us = my_range.merge(repo_us, on='sku')
repo_us.set_index('date', inplace = True)
display(repo_us.head())




ukdir = '/content/drive/My Drive/Sales_Data_Connected_Folder/Amz_Data/UK_ALL_ORDERS'
y = ['2021', '2022']
ukdf = all_orders(ukdir, y)
ukdf = cull(ukdf, 'Amazon.co.uk')
ukdf.sort_values('date', ascending=False, inplace=True)
uk_complete_df = cum(ukdf)

last_week  = landscape(usdf)
uk_min_max = get_min_max(uk_complete_df)
uk_my_range = last_week.merge(uk_min_max, on='sku')

repo_uk = create_report(uk_complete_df, 1)  # default is 1
repo_uk = uk_my_range.merge(repo_uk, on='sku')
repo_uk.set_index('date', inplace = True)
display(repo_us.head())

# repo_uk = create_report(uk_complete_df, 1)  # default is 1


"""Add to sheet"""
usurl ='https://docs.google.com/spreadsheets/d/1u2NbP3lkn6Unf6J9R_0wN76m3iOO4ElXi2-maxN8PLA/edit#gid=1976885912'
ukurl = 'https://docs.google.com/spreadsheets/d/1MZrAF6fbK3lJZseXge0Zo7cWvDJEHqEmAQGOYjMLP8U/edit#gid=0'
test = 'https://docs.google.com/spreadsheets/d/1nRCa_ZGTVjmXT1kM1FaFVooBji_SV2zhQ8L-2UbOs84/edit#gid=0'

# create_sheet(repo_us, test, mytitle='test1')
# create_sheet(df = repo_us, url=usurl, mytitle = 'amzshopify created')
# create_sheet(df = repo_uk,url=ukurl, mytitle = 'ukamz')


data import complete


daily  ...  weighted_avg_weekly_sales
date                      sku                       ...                           
2017-01-01 00:00:00+00:00 Toniiq-12-Packets      2  ...                        2.0
                          Toniiq-6-Packets       2  ...                        2.0

[2 rows x 10 columns]

,sku,2022-02-20,"(sum7days, min)","(sum7days, max)",avg7,sum7days,avg30,sum30days,avg90,sum90days,avg180,sum180days,weighted_avg_weekly_sales
date,,,,,,,,,,,,,
2022-02-27 00:00:00+00:00,5-HTP New,307.0,211.0,316.0,45.142857,316.0,16.115385,419.0,28.383721,2441.0,31.937500,5621.0,29.603760
2022-02-27 00:00:00+00:00,6S-B99O-0O0K,315.0,286.0,315.0,44.857143,314.0,42.633333,1279.0,39.233333,3531.0,41.505556,7471.0,42.113730
2022-02-27 00:00:00+00:00,AGPC7629,549.0,510.0,557.0,79.571429,557.0,74.333333,2230.0,70.367816,6122.0,76.974522,12085.0,75.179716
2022-02-27 00:00:00+00:00,AlphaLipoicAcid7728,416.0,257.0,502.0,47.714286,334.0,70.566667,2117.0,83.693182,7365.0,86.196629,15343.0,71.261193
2022-02-27 00:00:00+00:00,AppleCiderV7438,575.0,528.0,588.0,81.571429,571.0,105.433333,3163.0,77.844444,7006.0,66.527778,11975.0,84.789524


data import complete


,,daily,avg7,sum7days,avg30,sum30days,avg90,sum90days,avg180,sum180days,weighted_avg_weekly_sales
date,sku,,,,,,,,,,
2021-04-17 00:00:00+00:00,Quercetin7803,1,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
2021-04-18 00:00:00+00:00,Quercetin7803,3,2.0,4.0,2.0,4.0,2.0,4.0,2.0,4.0,2.0


,sku,2022-02-20,"(sum7days, min)","(sum7days, max)",avg7,sum7days,avg30,sum30days,avg90,sum90days,avg180,sum180days,weighted_avg_weekly_sales
date,,,,,,,,,,,,,
2022-02-27 00:00:00+00:00,5-HTP New,307.0,211.0,316.0,45.142857,316.0,16.115385,419.0,28.383721,2441.0,31.937500,5621.0,29.603760
2022-02-27 00:00:00+00:00,6S-B99O-0O0K,315.0,286.0,315.0,44.857143,314.0,42.633333,1279.0,39.233333,3531.0,41.505556,7471.0,42.113730
2022-02-27 00:00:00+00:00,AGPC7629,549.0,510.0,557.0,79.571429,557.0,74.333333,2230.0,70.367816,6122.0,76.974522,12085.0,75.179716
2022-02-27 00:00:00+00:00,AlphaLipoicAcid7728,416.0,257.0,502.0,47.714286,334.0,70.566667,2117.0,83.693182,7365.0,86.196629,15343.0,71.261193
2022-02-27 00:00:00+00:00,AppleCiderV7438,575.0,528.0,588.0,81.571429,571.0,105.433333,3163.0,77.844444,7006.0,66.527778,11975.0,84.789524


In [24]:
# display(repo_us.head())
display(us_complete_df.iloc[:-180])
display(us_complete_df.iloc[-180:])


daily  ...  weighted_avg_weekly_sales
date                      sku                                  ...                           
2017-01-01 00:00:00+00:00 Toniiq-12-Packets                 2  ...                   2.000000
                          Toniiq-6-Packets                  2  ...                   2.000000
                          Toniiq-Hydration-Concentrate      2  ...                   2.000000
2017-01-02 00:00:00+00:00 Toniiq-12-Packets                 9  ...                   5.500000
                          Toniiq-6-Packets                  5  ...                   3.500000
...                                                       ...  ...                        ...
2022-02-24 00:00:00+00:00 OliveLeaf120-7865                32  ...                  35.763968
                          PQQ7643                          58  ...                  61.137086
                          Phosphatidylserine7667           50  ...                  43.901587
                          Probiotic7964                   144  ...                 127.832286
                          Quercetin7377                   359  ...                 214.503366

[38823 rows x 10 columns]

daily  ...  weighted_avg_weekly_sales
date                      sku                            ...                           
2022-02-24 00:00:00+00:00 Resveratrol7322           297  ...                 275.975556
                          Spermidine7971              8  ...                   6.250000
                          St.John'sWort120 -7384     80  ...                  74.173651
                          TartCherry120-7391         46  ...                  39.327480
                          Tribulus7490              225  ...                 316.128651
...                                                 ...  ...                        ...
2022-02-28 00:00:00+00:00 St.John'sWort120 -7384     41  ...                  71.611746
                          TartCherry120-7391         10  ...                  37.892235
                          Tribulus7490               95  ...                 284.786667
                          TurkeyTail7940              8  ...                  23.854513
                          ValerianRoot7537           20  ...                  45.560089

[180 rows x 10 columns]

In [ ]:
create_sheet(repo_us, test, mytitle='test1')
create_sheet(df = repo_us, url=usurl, mytitle = 'amzshopify')
create_sheet(df = repo_uk,url=ukurl, mytitle = 'ukamz')

Updating existing worksheet: please check to make sure values are correct. If not, delete sheet and run again
Updating existing worksheet: please check to make sure values are correct. If not, delete sheet and run again
Updating existing worksheet: please check to make sure values are correct. If not, delete sheet and run again


In [10]:
"""Get predictions from the sheet that was created last"""

def get_pred(cumdf):
  """STILL WORKING ON THIS ONE"""
  cumdf.reset_index(inplace=True)
  cumdf.set_index('sku','date', inplace=True)
  gc = gspread.authorize(GoogleCredentials.get_application_default())
  # myurl = 'https://docs.google.com/spreadsheets/d/1nRCa_ZGTVjmXT1kM1FaFVooBji_SV2zhQ8L-2UbOs84/edit#gid=0'  # test sheet
  myurl = 'https://docs.google.com/spreadsheets/d/1u2NbP3lkn6Unf6J9R_0wN76m3iOO4ElXi2-maxN8PLA/edit#gid=1976885912'
  wb = gc.open_by_url(myurl)
  worksheet_list = wb.worksheets()

  from dateutil import parser
  def myparse(k):
    # print(k)
    try:
      return parser.parse(str(k), fuzzy=True).date()
      # return date
    except:
      return datetime.date.today() - datetime.timedelta(days=365)
      
  r= sorted([myparse(k) for k in worksheet_list], reverse = True)[0:4]

  selected_lst = [f'{p}amzshopify' for p in r]  # list of sheets we care about: 0 is last sheet created

  for mytitle in selected_lst:
    print(mytitle)

    sheet = wb.worksheet(mytitle)

    rows = sheet.get_all_values()
    df = pd.DataFrame.from_records(rows[1:], columns = rows[0])

    df.set_index(['date','sku'], inplace=True)
    display(df.head(1))
    pred_col = [col for col in df.columns if 'Gautam' in col]
    cumdf = cumdf.join(df[pred_col], how='outer')
  display(cumdf)
  return cumdf


cells = {"A": {1: 1, 2: 5, 3: 3},
          "B": {1: 2, 2: 6, 3: 3},
          "C": {1: 3, 2: 7, 3: 3},
          "D": {1: 4, 2: 8, 3: 3}}
dfnumbers = pd.DataFrame(cells)

# create_sheet(dfnumbers)

get_pred(repo_us)




2022-02-21amzshopify


,,"('sum7days', 'min')","('sum7days', 'max')",avg7,sum7days,avg30,sum30days,avg90,sum90days,avg180,sum180days,weighted_avg_weekly_sales,#N/A,#N/A,Gautam (Feb14),Gautam (Feb 22),Flag,% changed,Buffer to add %,New IMS Daily Sales,
date,sku,,,,,,,,,,,,,,,,,,,,
2022-02-21 00:00:00+00:00,Tribulus7490,2402,2689,366.8571429,2568,416.9333333,12508,288.4111111,25957,251.8166667,45327,339.2603968,,,400,350,Top 5,-12.50%,1.08,378,Tribulus7490


2022-02-14amzshopify


,,avg7,sum7days,avg30,sum30days,avg90,sum90days,avg180,sum180days,weighted_avg_weekly_sales,Gautam (Jan 23),Gautam (Jan31),Gautam (Feb7),Gautam (Feb14),Flag,% changed,Buffer to add %,New IMS Daily Sales
date,sku,,,,,,,,,,,,,,,,,
2022-02-13 00:00:00+00:00,Tribulus7490,425.4285714,2978,417.1666667,12515,274.5888889,24713,247.0722222,44473,349.5688095,400,400,400,400,Top 5,0.00%,1.08,432


ValueError: ignored

In [99]:
"""get list of week sum for 3 month data"""
from pandas.tseries.offsets import DateOffset

def create_report(mydf, time_d=1):
  # minmax = get_min_max(mydf)
  df = mydf.copy()
  df.reset_index(inplace=True)
  df.set_index('date', inplace=True)
  today = datetime.date.today() 
  pastday = today - datetime.timedelta(days=time_d)  # default is yesterday assuming folder was updated today
  date = str(pastday)
  finaldf = df.loc[date]
  # finaldf.sort_values('weekdiff', ascending=True, inplace=True)
  finaldf.drop('daily', axis=1, inplace=True)
  finaldf.reset_index(inplace = True)
 
  # finaldf = minmax.merge(finaldf, on='sku', left_index=True)

  return finaldf


def cum(mydf, i_freq = [7, 30, 90, 180]):
  """takes in xxdf and Returns xx_complete_df """
  # using day(), sum by defined frequency take in df and list of summation period frequencies: return sum by period, mean by period, and weighted averages
  df = day(mydf)
  byday = df.copy()
  byday.reset_index(inplace=True)
  # I think I can put the landscape sheet stuff here? I just need to figure out how to concat on sku: but the index is reset?

  byday.set_index(['date', 'sku'], inplace=True)
  byday.columns =  ['daily']

  for i in i_freq:

    mydf = df.groupby('sku')['quantity'].rolling(f'{i}D').sum().to_frame()
    mydf.reset_index(inplace=True)
    mydf.set_index(['date', 'sku'], inplace=True)
    mydf.columns = [f'sum{i}days']
    # display(mydf.head())

    mydf2 = df.groupby('sku')['quantity'].rolling(f'{i}D').mean().to_frame()
    mydf2.reset_index(inplace=True)
    mydf2.set_index(['date', 'sku'], inplace=True)
    mydf2.columns = [f'avg{i}']

    byday = byday.join([mydf2, mydf], how='outer')

  byday.fillna(0, inplace=True)
  #  complete_cum['weighted_avg_daily_sales'] = ((complete_cum['daily']*.3) + ((complete_cum['weekly']/7)*.3) + ((complete_cum['monthly']/30)*.3) + ((complete_cum['6_mon']/(30*6))*.1))
  byday['weighted_avg_weekly_sales'] = (((byday['avg7'])*.25) + ((byday['avg30'])*.30) + ((byday['avg90'])*.25) + ((byday['avg180'])*.20))
  display(byday.head(2))
  
  return byday

def get_min_max(df):
  """Takes in the complete_cum data (create report)"""
  
  mydf = df.copy()
  date_df = mydf.iloc[-180:]
  date_df.reset_index(inplace=True)
  date_df.set_index('date', inplace = True)
  date_df['weekday'] = date_df.index.weekday
  new = date_df.groupby(['sku']).agg({'sum7days': ['min', 'max']})
  return(new)


def landscape(mydf):
  """Takes in xxdf"""
  mydf.sort_values('date', inplace=True)
  mydf['week_delta'] = pd.to_datetime(mydf['date']) - pd.to_timedelta(6, unit='d') # set time delta

  cum_weekly = mydf.groupby('sku').resample('W', on='week_delta').sum()
  cum_weekly.sort_values('week_delta', inplace=True)
  cum_weekly.reset_index(inplace=True)
  cum_weekly['week_delta'] = cum_weekly['week_delta'].dt.date
  
  weekly_df = cum_weekly.pivot(index='sku', columns='week_delta', values='quantity')
  weekly_df = weekly_df.iloc[: ,-2 :-1]

  # get minimum maximum
  cum_weekly.set_index('week_delta', inplace=True)

  min_max = cum_weekly[cum_weekly.last_valid_index()-DateOffset(180, 'D'):cum_weekly.last_valid_index()-DateOffset(1, 'D')]
  min_max.reset_index(inplace=True)
  min_max['week_delta'] = pd.to_datetime(min_max['week_delta'])
  min_max = min_max.groupby(['sku']).agg({'quantity': ['min', 'max']})

  min_max.reset_index(inplace=True)
  weekly_df.reset_index(inplace=True)

  new = weekly_df.merge(min_max, on = 'sku')

  return new

last_week  = landscape(usdf)

display(last_week)
# min_max = get_min_max(us_complete_df)

# my_range = last_week.merge(min_max, on='sku')

# repo_us = create_report(us_complete_df, 1)


# repo_us = my_range.merge(repo_us, on='sku')
# repo_us.set_index('date', inplace = True)


# display(repo_us.head())


,sku,2022-02-20,"(quantity, min)","(quantity, max)"
0,5-HTP New,307.0,6,307
1,6S-B99O-0O0K,315.0,184,382
2,AGPC7629,549.0,0,731
3,AlphaLipoicAcid7728,416.0,305,752
4,AppleCiderV7438,575.0,314,943
5,AshwagandhaPlus7605,709.0,368,842
6,Astragalus7551,80.0,0,156
7,Aswagandha7421,686.0,611,1280
8,Bacopa7445,196.0,0,198
9,BeetRoot7483,656.0,435,805
